In [1]:
# Import necessary libraries

In [1]:
pip install --upgrade tensorflow pandas numpy


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.8 kB 388.9 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 460.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.7 kB 653.6 kB/s eta 0:00:01
     -------------------------------------- 59.7/59.7 kB 785.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.6 MB 3.2 MB/s eta 0:00:04
   ---------------------------------------- 0.1/11.6 MB 1.7 MB/s eta 0:00:07
    --------------------------------------- 0.2/11.6 MB 1.4 MB/s eta 0:00:09
    --------------------------------------- 0.2/11.6 MB 1.3 MB/s eta 0:00:10
    ---------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.2 which is incompatible.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.0.2 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.0.2 which is incompatible.
numba 0.59.0 requires numpy<1.27,>=1.22, but you have numpy 2.0.2 which is incompatible.
pywavelets 1.5.0 requires numpy<2.0,>=1.22.4, but you have numpy 2.0.2 which is incompatible.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.0.2 which is incompatible.
streamlit 1.30.0 requires numpy<2,>=1.19.3, but you have numpy 2.0.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
import mysql.connector
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Connect to MySQL database

In [ ]:
conn = mysql.connector.connect(
    host='18.136.157.135',
    user='dm_team1',
    password='DM!$Team&279@20!',
    database='project_banking'
)

In [ ]:
# Load data from SQL tables

In [ ]:
query_account = 'SELECT * FROM Cust_Account'
query_enquiry = 'SELECT * FROM Cust_Enquiry'
query_demographics = 'SELECT * FROM Cust_Demographics'
df_account = pd.read_sql(query_account, conn)
df_enquiry = pd.read_sql(query_enquiry, conn)
df_demographics = pd.read_sql(query_demographics, conn)
conn.close()

In [ ]:
# Merge datasets

In [ ]:
merged_df = df_account.merge(df_enquiry, on='customer_no', how='left')
merged_df = merged_df.merge(df_demographics, on='customer_no', how='left')
merged_df.fillna(0, inplace=True)

In [ ]:
# Convert date columns to datetime format

In [ ]:
date_columns = ['dt_opened_x', 'opened_dt', 'last_paymt_dt', 'closed_dt', 'reporting_dt', 'paymt_str_dt', 'paymt_end_dt', 'dt_opened_y', 'upload_dt_y', 'enquiry_dt']
for col in date_columns:
    merged_df[col] = pd.to_datetime(merged_df[col], errors='coerce')

In [ ]:
# Data Exploration

In [ ]:
print('Data Overview:')
print(merged_df.info())
print(merged_df.describe())

In [ ]:
# Feature Engineering

In [ ]:
merged_df['credit_utilization'] = merged_df['cur_balance_amt'] / (merged_df['high_credit_amt'] + 1)
merged_df['payment_gap'] = (merged_df['last_paymt_dt'] - merged_df['opened_dt']).dt.days
merged_df['recent_enquiry_count'] = merged_df.groupby('customer_no')['enquiry_dt'].transform(lambda x: x.count())

In [ ]:
# Convert numeric columns to proper types
numeric_cols = ['cur_balance_amt', 'high_credit_amt']
merged_df[numeric_cols] = merged_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Feature Selection

In [ ]:
features = ['credit_utilization', 'payment_gap', 'recent_enquiry_count']
target = 'Bad_label'
X = merged_df[features]
y = merged_df[target]

In [ ]:
# Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train multiple models

In [ ]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42),
    'Logistic Regression': LogisticRegression()
}

In [ ]:
model_performance = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    roc_score = roc_auc_score(y_test, y_pred_proba)
    gini_score = 2 * roc_score - 1
    model_performance[name] = gini_score

In [ ]:
# Identify best model

In [ ]:
best_model = max(model_performance, key=model_performance.get)

In [ ]:
# Make final predictions using best model

In [ ]:
best_model_instance = models[best_model]
y_pred_proba = best_model_instance.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

In [ ]:
# Print evaluation results

In [ ]:
print('Model Performance (Gini Scores):', model_performance)
print('Best Model:', best_model)

In [ ]:
# Feature Importance

In [ ]:
if best_model in ['Random Forest', 'Gradient Boosting']:
    feature_importance = pd.DataFrame({'Feature': features, 'Importance': best_model_instance.feature_importances_}).sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(8, 6))
    sns.barplot(x=feature_importance['Importance'], y=feature_importance['Feature'])
    plt.title(f'Feature Importance - {best_model}')
    plt.show()

In [ ]:
# Print classification results

In [ ]:
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))

In [ ]:
# Final Conclusion

In [ ]:
print('\nFinal Conclusion:')
print('1. The best-performing model is', best_model, 'with a Gini Score of', model_performance[best_model], '.')
print('2. The key factors influencing credit risk are Credit Utilization, Payment Gap, and Recent Enquiry Count.')
print('3. This model helps in predicting high-risk customers, allowing Bank GoodCredit to reduce credit default risk.')
print('4. The results can be used for better decision-making in credit approvals and risk management.')